In [1]:
// place a csv file names simple_data.csv in the same folder as this notebook 
// file contents:
//
//id,data
//100, {'name':'037717956'}
//200, {'name':'077564444'}

org.apache.spark.sql.SparkSession@52f04e96

In [59]:
case class SimpleData(name: String)
val simpleJsonStruct = Encoders.product[SimpleData].schema
simpleJsonStruct

StructType(StructField(name,StringType,true))

In [64]:
val simpleDF = spark.read.option("header","true").csv("simple_data.csv")

In [65]:
simpleDF.printSchema()
simpleDF.show()

root
 |-- id: string (nullable = true)
 |-- data: string (nullable = true)

+---+--------------------+
| id|                data|
+---+--------------------+
|100| {'name':'0377179...|
|200| {'name':'0775644...|
+---+--------------------+



In [69]:
val parsedDF = simpleDF.withColumn("data2", from_json(col("data"), simpleJsonStruct))
parsedDF.show()

+---+--------------------+-----------+
| id|                data|      data2|
+---+--------------------+-----------+
|100| {'name':'0377179...|[037717956]|
|200| {'name':'0775644...|[077564444]|
+---+--------------------+-----------+



In [70]:
parsedDF.printSchema()

root
 |-- id: string (nullable = true)
 |-- data: string (nullable = true)
 |-- data2: struct (nullable = true)
 |    |-- name: string (nullable = true)



In [72]:
parsedDF.select("data2.name").show()

+---------+
|     name|
+---------+
|037717956|
|077564444|
+---------+



In [37]:
val dataDF1 = spark.read.option("header","true").csv("medication_schedule_history_all_mjff.csv")

In [38]:
dataDF1.show(10)

+----------+-------+-------+-------------------+-------------------+--------------------+
|experiment| cohort|user_id|schedule_time_start|  schedule_time_end| medication_schedule|
+----------+-------+-------+-------------------+-------------------+--------------------+
|Clinicians|alabama| 142557|2017-06-15 13:29:38|2017-06-15 13:30:04|"{""schedule"":{"...|
|Clinicians|alabama| 142557|2017-06-15 13:30:04|2017-06-15 13:30:28|"{""schedule"":{"...|
|Clinicians|alabama| 142557|2017-06-15 13:30:28|2017-06-15 13:30:58|"{""schedule"":{"...|
|Clinicians|alabama| 142557|2017-06-15 13:30:58|2017-06-15 13:31:33|"{""schedule"":{"...|
|Clinicians|alabama| 142557|2017-06-15 13:31:33|2017-06-23 11:06:36|"{""schedule"":{"...|
|Clinicians|alabama| 142557|2017-06-23 11:06:36|2017-06-23 11:06:50|"{""schedule"":{"...|
|Clinicians|alabama| 142557|2017-06-23 11:06:50|2017-06-23 11:06:59|"{""schedule"":{"...|
|Clinicians|alabama| 142557|2017-06-23 11:06:59|2017-06-24 12:23:21|"{""schedule"":{"...|
|Clinician

In [33]:
dataDF.cache()

[experiment: string, cohort: string ... 4 more fields]

In [34]:
import scala.util.parsing.json._
import org.apache.spark.sql.functions._

case class MedicationScheduleRow(name: String, quantity: Double, quantity_type: String)
case class MedicationScheduleClass(schedule : Map[String,List[MedicationScheduleRow]])
case class TableData(id : Long, version : Integer, user_id: Long,schedule_time : String, medication_schedule: MedicationScheduleClass)
val medicationScheduleJsonStruct = Encoders.product[MedicationScheduleClass].schema

val jsonDF = dataDF.withColumn("medication_schedule", from_json(col("medication_schedule"), medicationScheduleJsonStruct))

In [36]:
jsonDF.take(1)

Array([Clinicians,alabama,142557,2017-06-15 13:29:38,2017-06-15 13:30:04,null])

In [35]:
dataDF.select("medication_schedule").take(5)

Array(["{""schedule"":{""660"":[{""name"":""Sinemet""], ["{""schedule"":{""660"":[{""name"":""Sinemet""], ["{""schedule"":{""420"":[{""name"":""Sinemet""], ["{""schedule"":{""420"":[{""name"":""Sinemet""], ["{""schedule"":{""420"":[{""name"":""Sinemet""])